In [1]:
from collections import Counter
import json
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
import re

#### Veel stukken van deze code zijn overgenomen van Joost en Max

In [5]:
def preprocess(article):
    article = article.encode('utf-8')
    article = article.lower()
    regex = re.compile('[^a-z\s]')
    article = regex.sub('', article.decode('utf-8'))
    return article

In [13]:
path = r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_topics_first.json"

with open(path) as data_file:    
    data = json.load(data_file)
    
categories = []
content = []

for article in data:
    category = article["category"]#.split("|")
    categories.append(category)#[0])[0])
    content.append(preprocess(article["content"]))
    
path = r"C:\Users\PC-Axel\Documents\github\thesis\Data\PoliFLW Data\kamerstukken_topics_second.json"

with open(path) as data_file:    
    testData = json.load(data_file)

for article in testData:
    category = article["category"]#.split("|")
    categories.append(category)#[0])
    content.append(preprocess(article["content"]))
    
print(len(content))
Counter(categories)

28294


Counter({'Bestuur | De Nederlandse Antillen en Aruba': 189,
         'Bestuur | Gemeenten': 431,
         'Bestuur | Koninklijk Huis': 62,
         'Bestuur | Organisatie en beleid': 294,
         'Bestuur | Parlement': 86,
         'Bestuur | Provincies': 28,
         'Bestuur | Rijksoverheid': 136,
         'Bestuur | Waterschappen': 30,
         'Cultuur en recreatie | Cultuur': 111,
         'Cultuur en recreatie | Kunst': 33,
         'Cultuur en recreatie | Media': 243,
         'Cultuur en recreatie | Organisatie en beleid': 51,
         'Cultuur en recreatie | Recreatie': 52,
         'Cultuur en recreatie | Religie': 375,
         'Cultuur en recreatie | Sport': 255,
         'Economie | Bouwnijverheid': 61,
         'Economie | Handel': 368,
         'Economie | ICT': 297,
         'Economie | Industrie': 89,
         'Economie | Markttoezicht': 159,
         'Economie | Ondernemen': 282,
         'Economie | Organisatie en beleid': 200,
         'Economie | Overige economisc

In [14]:
transformer = TfidfVectorizer(smooth_idf=True, min_df=0.00000001, max_df=0.2, sublinear_tf=True)

X_train_corpus, X_test_corpus, y_train, y_test = train_test_split(content, categories, test_size=0.2, random_state=42)

X_train = transformer.fit_transform(X_train_corpus)
X_test = transformer.transform(X_test_corpus)

print(np.shape(X_train))
print(np.shape(y_test))

(22635, 147555)
(5659,)


In [17]:
clf = SGDClassifier(loss='log', penalty='l1', alpha=1e-6, random_state=42, max_iter=10, tol=None)
clf = OneVsRestClassifier(clf).fit(X_train, y_train)

y_pred = clf.predict(X_test)

proba = clf.predict_proba(X_test[0])
print(len(proba[0]))

print(y_pred)
score = accuracy_score(y_test, y_pred)

print(score)

112
['Natuur en milieu | Natuur- en landschapsbeheer' 'Verkeer | Weg'
 'Landbouw | Dieren' ...
 'Openbare orde en veiligheid | Organisatie en beleid'
 'Internationaal | Ontwikkelingssamenwerking'
 'Internationaal | Internationale samenwerking']
0.7709842728397244


In [16]:
clf = MultinomialNB()
clf = OneVsRestClassifier(clf).fit(X_train, y_train)

y_pred = clf.predict(X_test)

proba = clf.predict_proba(X_test[0])
print(len(proba[0]))

score = accuracy_score(y_test, y_pred)

print(score)

112
0.1851917299876303


In [9]:
from sklearn.metrics import f1_score, precision_score, recall_score


print(f1_score(y_test, y_pred, average='macro'),f1_score(y_test, y_pred, average='micro'), 
      f1_score(y_test, y_pred, average='weighted') )

print(precision_score(y_test, y_pred, average='macro'),precision_score(y_test, y_pred, average='micro'), 
      precision_score(y_test, y_pred, average='weighted') )

print(recall_score(y_test, y_pred, average='macro'),recall_score(y_test, y_pred, average='micro'), 
      recall_score(y_test, y_pred, average='weighted') )

(0.7339813373582283, 0.7702774341756494, 0.7678165826171034)
(0.7767031198079153, 0.7702774341756494, 0.7741080830934394)


C:\Users\Axel\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.7168221446155666, 0.7702774341756494, 0.7702774341756494)
